In [ ]:
# Run this notebook with access to the `strawb.Config.pandas_file_sync_db` file
# i.e. on a LRZ vm with mounted `dss`
# or via downloading `/dss/strawb/raw_module_data/pandas_file_sync_db.gz` to your local machine
# >>>  rsync --progress -au straw-lrz-vm:"/dss/strawb/raw_module_data/pandas_file_sync_db.gz" .
# and adopted 'pandas.read_pickle(strawb.Config.pandas_file_sync_db)' accordingly

import strawb

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

import pandas

import numpy as np
import datetime

# set plotly default to plotly_white and define a presonal colorway
pio.templates["myname"] = go.layout.Template(
    layout=go.Layout(
        colorway=px.colors.qualitative.T10,  # colors
        yaxis = dict(exponentformat = 'SI'),  # exponentformat to SI, i.e., 1e9 -> 1G
    ))
pio.templates.default = "plotly_white+myname"  # 'plotly_white' and overwrite with the pramters in 'myname'

# Load the DataFrame and add a `deviceName`

In [ ]:
db = strawb.SyncDBHandler(file_name='Default')
db.dataframe

# add a new column to simplify the plotting
db.dataframe['deviceName'] = db.dataframe['deviceCode'].str.replace('TUM','').str.replace('00',' ')

In [ ]:
db.dataframe

## File size by deviceCode

In [ ]:
# Mask data
# mask = [pd_sync['fileSize'] > 5e8],  # mask by filesSize 
mask = db.dataframe['dataProductCode']=='LIDARTOT'  # only a specific 'dataProductCode'
# mask = pd_sync['dataProductCode']!=''  # all data

fig_1 = px.scatter(db.dataframe[mask],
                   x='dateFrom', y='fileSize',
                   color='deviceName',
                   hover_data=['filename', 'dataProductCode'],
                   opacity=.5,
                  )
fig_1.show()

## File size by dataProductCode for the LiDAR001 only

remove `[pd_sync['deviceCode'] == 'TUMLIDAR001']` for to show all modules

In [ ]:
# Mask
mask = db.dataframe['deviceCode'] == 'TUMLIDAR001'  # files from the LIDAR1 only
mask |= db.dataframe['deviceCode'] == 'TUMLIDAR002'  # and add files from the LIDAR2

fig_2 = px.scatter(db.dataframe[mask],
                   x='dateFrom', y='fileSize',
                   color='dataProductCode',
                   hover_data=['filename', 'deviceCode'],
                   opacity=.5,
                  )
fig_2.show()

## Explore hdf5 meta data, aka. hdf5 attributes

In [ ]:
# Mask
mask = db.get_mask_h5_attrs()

fig_2 = px.scatter(db.dataframe[mask],
                   x='dateFrom', y='fileSize',
                   color='dataProductCode',
                   hover_data=['filename', 'deviceCode', 'measurement_type'],
                   opacity=.5,
                  )
fig_2.show()

## Total File Size per Day and Module

In [ ]:
# cal. pins, each day one bin
bins = pandas.date_range(start=db.dataframe['dateFrom'].min(),
                         end=db.dataframe['dateFrom'].max() #+ datetime.timedelta(days=1)
                        ).to_numpy('datetime64')

# cal. for each module the total file size per day (bin)
size_per_day = {}
for dev_i in db.dataframe['deviceName'].unique():
    pd_dev_i = db.dataframe[db.dataframe['deviceName']==dev_i]
    
    # exclude log file, as this is data not produced by the module rather than the ONC daq system
    pd_dev_i = pd_dev_i[pd_dev_i['dataProductCode'] != 'LF']
    hist, bin_edges = np.histogram(pd_dev_i['dateFrom'].to_numpy('datetime64'),
                                   bins=bins,
                                   weights=pd_dev_i['uncompressedFileSize'])
    
    size_per_day[dev_i] = hist

# convert it to a DataFrame
size_per_day = pandas.DataFrame(size_per_day)
size_per_day.index = bins[:-1]  # set day as index of DataFrame

In [ ]:
# create a scatter plot, for the trendlines see https://plotly.com/python/linear-fits/
fig = px.scatter(size_per_day,
                 # trendline="lowess", trendline_options=dict(frac=0.1),
                 trendline='rolling', trendline_options=dict(window=7),
                 #marginal_y="violin",
                 #log_y=True,
                 #opacity=.5,
                 labels={
                     "index": "Date",
                     "value": "File Size per Day (Byte)",
                     "variable": "Module",
                 },
                 range_y=(size_per_day[size_per_day!=0].min().min()*.9,
                          size_per_day[size_per_day!=0].max().max()*1.1),
                )
fig.show()

In [ ]:
# create a line plot
fig = px.line(size_per_day, y=size_per_day.keys(), log_y=True, #range_x=[300e3, ]
              #markers=True,
              range_y=(49e3, #size_per_day[size_per_day!=0].min().min()*.9,
                       size_per_day[size_per_day!=0].max().max()*1.1),  # s_numpy[s_numpy>0].min()*.9
              #line_shape="spline",  # 'linear' or "spline"
              labels={
                  "index": "Date",
                  "value": "File Size per Day (Byte)",
                  "variable": "Module",
              },
              render_mode="svg")

# fig.update_layout({'xaxis': {'linecolor': 'rgb(36,36,36)',
#                              'showline': True,
#                              'ticks': 'outside'},
#                    'yaxis': {'linecolor': 'rgb(36,36,36)',
#                              'showline': True,
#                              'ticks': 'outside'}
#                   })


fig.show()

In [ ]:
db.dataframe.columns